In [1]:
import numpy as np
import pandas as pd
from dijkstra import DijkstraSPF, Graph
from sklearn.neighbors import BallTree
import math
import csv
import time

In [2]:
data_file = './data/worldcities_ascii.csv'
with open(data_file, 'r') as f:
    reader = csv.reader(f)
    all_lines = []
    for row in reader:
        all_lines.append(row)

In [3]:
newLines = []
for line in all_lines:
    newLine = []
    for index, field in enumerate(line):
        if ("." in field) and (index == 5): #Toglie il '.' nei valori della popolazione
            field = field.replace(".", "")
        field = field.replace("Korea, South", "South Korea")
        field = field.replace("Korea, North", "North Korea")
        field = field.replace("Gambia, The", "The Gambia")
        field = field.replace("Micronesia, Federated States Of", "Federated States Of Micronesia")
        field = field.replace("Bahamas, The", "The Bahamas")
        field = field.replace("Saint Helena, Ascension, And Tristan Da Cunha", "Saint Helena, Ascension and Tristan da Cunha")
        field = field.replace("Islamorada, Village of Islands", "Village of Islands Islamorada")
        if index == 6:
            field = field.replace("\n", "")
        newLine.append(field)
    newLines.append(newLine)

#Lo esporto in caso mi servisse in altri progetti
df = pd.DataFrame(newLines[1:], columns=newLines[0])
#df["population"].replace({"": -1}, inplace= True) #Correggo alcuni valori che non hanno la popolazione con 0 anche se sarebbe più corretto lasciare null/None
df = df.astype({"lat": float, "lng": float, "id": int})
#df.to_csv("./data/worldcities_preprocessed.csv", index=False)
df

,city,lat,lng,country,iso3,population,id
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077
2,Delhi,28.6600,77.2300,India,IND,29617000,1356872604
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140
...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8000,Greenland,GRL,10,1304217709
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438
26567,Ambarchik,69.6510,162.3336,Russia,RUS,0,1643739159


## Mi costruisco il grafo con un balltree

In [4]:
coords = [np.array([c.lat, c.lng])*math.pi/180 for _, c in df.iterrows()]

In [5]:
tree = BallTree(coords, leaf_size=4, metric="haversine")

In [6]:
def distanceTime(start, end, neighbour_pos: int):
    if neighbour_pos == 0:
        time = 2
    elif neighbour_pos == 1:
        time = 4
    elif neighbour_pos == 2:
        time = 8
    if start.iso3 != end.iso3:
        time += 2
    if (end.population != '') and (int(end.population) > 200000):
        time += 2
    return time

In [7]:
graph = Graph()
neighbours = list()
for i in df.index:
    n = tree.query(coords[i:i+1], return_distance=False, breadth_first=True, k=4)[0,1:]
    
    graph.add_edge(i, n[0], distanceTime(df.iloc[i], df.iloc[n[0]], 0))
    graph.add_edge(i, n[1], distanceTime(df.iloc[i], df.iloc[n[1]], 1))
    graph.add_edge(i, n[2], distanceTime(df.iloc[i], df.iloc[n[2]], 2))
    
    neighbours.append(n)
df_neigh = df.assign(neighbours=neighbours)

In [8]:
startNode = 3175 #Città di partenza
dijkstra = DijkstraSPF(graph, startNode)
print("%-5s %-5s %-10s" % ("label", "distance", "path"))
for endNode in df_neigh.index:
    dist = dijkstra.get_distance(endNode)
    if dist != float('inf'):
        print("%-5s %-8d %-10s" % (endNode, dist, dijkstra.get_path(endNode)))
        break

label distance path      
2     108      [3175, 19004, 14093, 14599, 10956, 831, 14757, 2913, 253, 1810, 2275, 10809, 3809, 627, 15303, 3178, 2520, 2709, 337, 2]


# Map ipyleaflet

In [9]:
from ipyleaflet import Map, basemaps, Marker, Polyline, TileLayer, WidgetControl, FullScreenControl, AwesomeIcon
from ipywidgets import Combobox

## All cities neighbours

Vedere se si riesce a risolvere la questione della combobox (si potrebbe fare dizionario che codifica solo doppioni)

In [10]:
class MapWrapper:
    
    def __init__(self, mapInstance: Map, dataset: pd.DataFrame, cityList: Combobox):
        self.map = mapInstance
        self.map.layout.height = "510px"
        self.df = dataset
        self.cityList = cityList
        
        self.map.add_control(WidgetControl(widget=self.cityList, position='topright'))
        self.map.add_control(FullScreenControl())

        def callback(args):
            if args["new"] == "":
                return
            self.clear_map()
            index = int(args["new"].split("- ")[-1])
            city = self.df.iloc[index]
            self.map.center = (city.lat, city.lng)
            self.map.zoom = 8
            self.draw_neighbours(city)
            self.cityList.value = ""

        self.cityList.observe(callback, names="value")
        
    def draw_marker(self, city, icon=AwesomeIcon()):
        marker = Marker(location=(city.lat, city.lng), draggable=False, title=city.city, icon=icon)
        #marker.on_click(lambda **a: add_marker(df.iloc[193]))
        self.map.add_layer(marker)
    
    def draw_line(self, coords, color="green"):
        line = Polyline(
            locations=coords,
            color=color,
            fill=False
        )
        self.map.add_layer(line)

    def draw_neighbours(self, city):
        self.draw_marker(city)
        for neigh_index in city.neighbours:
            neigh = self.df.iloc[neigh_index]
            self.draw_marker(neigh, AwesomeIcon(name='circle', marker_color='green'))
            coords = [
                [city.lat, city.lng],
                [neigh.lat, neigh.lng]
            ]
            self.draw_line(coords)

    def clear_map(self):
        for layer in self.map.layers:
            if type(layer) is not TileLayer:
                self.map.remove_layer(layer)
    
    def display(self):
        display(self.map)

In [11]:
options = [f"{city.city} ({city.iso3}) - {i}" for i, city in df.iterrows()]

In [12]:
m = Map(basemap=basemaps.OpenStreetMap.HOT, zoom=1, scroll_wheel_zoom=True, world_copy_jump=True)
cityList = Combobox(options=options, placeholder='ex. London', description='City:', ensure_option=True)
map1 = MapWrapper(m, df_neigh, cityList)
map1.display()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Viaggiare Londra-Londra andando solo a est

In [13]:
eastGraph = Graph()
neighbours = list()
for i in df.index:
    n = tree.query(coords[i:i+1], return_distance=False, breadth_first=True, k=4)[0,1:]
    eastNeigh = []
    
    for neighIndex in range(len(n)):
        if (df.iloc[i].lng < df.iloc[n[neighIndex]].lng) or ((df.iloc[i].lng > 0) and (df.iloc[n[neighIndex]].lng < 0) and (df.iloc[i].lng-df.iloc[n[neighIndex]].lng >= 180)):
            eastGraph.add_edge(i, n[neighIndex], distanceTime(df.iloc[i], df.iloc[n[neighIndex]], neighIndex))
            eastNeigh.append(n[neighIndex])
    
    neighbours.append(eastNeigh)
df_east = df.assign(neighbours=neighbours)
df_east

,city,lat,lng,country,iso3,population,id,neighbours
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764,"[4105, 1196]"
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077,[336]
2,Delhi,28.6600,77.2300,India,IND,29617000,1356872604,"[337, 523]"
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629,"[13002, 563, 946]"
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140,"[1497, 3456]"
...,...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8000,Greenland,GRL,10,1304217709,[]
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491,[]
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438,[14152]
26567,Ambarchik,69.6510,162.3336,Russia,RUS,0,1643739159,[25455]


In [14]:
m = Map(basemap=basemaps.OpenStreetMap.HOT, zoom=1, scroll_wheel_zoom=True, world_copy_jump=True)
cityList = Combobox(options=options, placeholder='ex. London', description='City:', ensure_option=True)
map2 = MapWrapper(m, df_east, cityList)
map2.display()

#Provare che non si può fare giro del mondo da nessuna città

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…